# Data Description

For this competition, you will be predicting a categorical target based on a number of feature columns given in the data. The data is synthetically generated by a GAN that was trained on a the data from the Forest Cover Type Prediction. This dataset is (a) much larger, and (b) may or may not have the same relationship to the target as the original data.

Please refer to this data page for a detailed explanation of the features.

Files
* train.csv - the training data with the target Cover_Type column
* test.csv - the test set; you will be predicting the Cover_Type for each row in this file (the target integer class)
* sample_submission.csv - a sample submission file in the correct format


*******From the competition data page.***************


In [ ]:
#From the discussion board (https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/291823)

!pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from functools import partial
import optuna
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import StratifiedKFold

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import gc
from scipy import stats

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')

In [ ]:
# from the discussion board (https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/291844)
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#df_train = reduce_mem_usage(df_train)
#df_test = reduce_mem_usage(df_test)

# 1. Data Explanatory Analysis and Cleaning

This is my third time to join Tabular Play Ground Series. I did not put much time for data explanatory analysis in the last two competitions. I will put some more effort on it this time to do effective feature engineering later on. To do so, I referred to [Machine Learning Explainability](https://www.kaggle.com/learn/machine-learning-explainability) course on Kaggle. 


In [ ]:
df_train.head(5)

In [ ]:
#nan values in dataframe?
df_train.isna().sum()

In [ ]:
df_train.dtypes

In [ ]:
def is_categorical(data,column):
    if len(data[column].unique()) <= 15:
        print(str(column) + ": " + str(data[column].unique()))
    return None
    
    

In [ ]:
columns = df_train.columns.to_list()
for column in columns:
    is_categorical(df_train,column)
    

In [ ]:
#Soil_Type7 and SoilType15 has only zero values. Need to delete those two columns.

df_train = df_train.drop(['Soil_Type7','Soil_Type15'],axis=1)
df_test = df_test.drop(['Soil_Type7','Soil_Type15'],axis=1)


In [ ]:
df_train.tail(10)

In [ ]:
# Cover_Type 5 was only one sample in this data.  
df_train = df_train[df_train.Cover_Type != 5]

In [ ]:
targets = df_train.Cover_Type
features = df_train.drop(['Cover_Type'],axis=1)



In [ ]:
#Extra feature engineering from the discussion board https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/293373

features["Aspect"][features["Aspect"] <0] +=360
features["Aspect"][features["Aspect"] >359]-=360

df_test["Aspect"][df_test["Aspect"] <0] +=360
df_test["Aspect"][df_test["Aspect"] >359] -=360


features.loc[df_train["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
df_test.loc[df_test["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

features.loc[df_train["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
df_test.loc[df_test["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0

features.loc[df_train["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
df_test.loc[df_test["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0

features.loc[df_train["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
df_test.loc[df_test["Hillshade_9am"] > 255, "Hillshade_9am"] = 255

features.loc[df_train["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
df_test.loc[df_test["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255

features.loc[df_train["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
df_test.loc[df_test["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255



In [ ]:
#extra feature engineering from the discussion board. 
#sum of soil_type and wilderness_ares https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/292823


feature_list = features.columns
soil_features = [x for x in feature_list if x.startswith("Soil_Type")]
features['soil_type_count'] = features[soil_features].sum(axis=1)
df_test['soil_type_count'] =df_test[soil_features].sum(axis=1)

wilderness_features= [x for x in feature_list if x.startswith('Wilderness')]
features['wilderness_area_count']=features[wilderness_features].sum(axis=1)
df_test['wilderness_area_count'] = df_test[wilderness_features].sum(axis=1)

In [ ]:
#some more features engineering
# from this discussion https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/293612


features['Euclidean_Distance_to_Hydrology'] =  ((features['Horizontal_Distance_To_Hydrology'])**2 + (features['Vertical_Distance_To_Hydrology'])**2)**0.5


features['Manhattan_Distance_to_Hydrology'] = np.abs(features['Horizontal_Distance_To_Hydrology']) + np.abs(features['Vertical_Distance_To_Hydrology'])


df_test['Euclidean_Distance_to_Hydrology'] =  ((df_test['Horizontal_Distance_To_Hydrology'])**2 + (df_test['Vertical_Distance_To_Hydrology'])**2)**0.5

df_test['Manhattan_Distance_to_Hydrology'] = np.abs(df_test['Horizontal_Distance_To_Hydrology']) + np.abs(df_test['Vertical_Distance_To_Hydrology'])

In [ ]:
#Scaling the values.


from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
#scaler = preprocessing.StandardScaler()
#scaler = preprocessing.RobustScaler()

numeric_features = features.columns[1:11].to_list()

for i in ['soil_type_count',
       'wilderness_area_count', 'Euclidean_Distance_to_Hydrology',
       'Manhattan_Distance_to_Hydrology']:
    numeric_features.append(i)
     


features[numeric_features] = scaler.fit_transform(features[numeric_features])
df_test[numeric_features] = scaler.transform(df_test[numeric_features])

In [ ]:

encoder = LabelEncoder()
targets[:] = encoder.fit_transform(targets[:])

In [ ]:
del df_train

In [ ]:
#train_X,val_X,train_y,val_y = train_test_split(features,targets,random_state=1)


# Making Model and Predict 


In [ ]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

def objective(trial,X,y, name='xgb'):
    params = param = {
        'objective':'multi:softmax',
        'tree_method':'gpu_hist',  
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical(
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.25,0.3,0.35,0.4,0.2,0.1,0.09, 0.01]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300, 3000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,15,17]
        ),
        'random_state': 42,
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1, 300
        ),
        'eval_metric':'auc',
        #'num_of_classes':4,
        
    }

    model =  XGBClassifier(**params)
    model.fit(train_X,train_y,eval_set=[(val_X,val_y)],early_stopping_rounds=50,verbose=False)


    train_score = np.round(accuracy_score(train_y, model.predict(train_X)), 5)
    test_score = np.round(accuracy_score(val_y, model.predict(val_X)), 5)
                  
    print(f'TRAIN ROC : {train_score} || TEST ROC : {test_score}')
                  
    return test_score

In [ ]:
#%%time
#optimize = partial(objective,X=train_X,y=train_y)

#study_lgbm = optuna.create_study(direction ='maximize')
#study_lgbm.optimize(optimize,n_trials=30)


In [ ]:
#print(f"\tBest value (roc): {study_lgbm.best_value:.5f}")
#print(f"\tBest params:")

#for key, value in study_lgbm.best_params.items():
#   print(f"\t\t{key}: {value}")


#Best value (roc): 0.96218
#	Best params:
#		lambda: 1.582390097702773
#		alpha: 0.00641014056102776
#		colsample_bytree: 0.8
#		subsample: 1.0
#		learning_rate: 0.09
#		n_estimators: 3000
#		max_depth: 13
#		min_child_weight: 39



In [ ]:
params = {
'lambda': 1.582390097702773,
'alpha': 0.00641014056102776,
'colsample_bytree': 0.8,
'subsample': 1.0,
'learning_rate': 0.09,
'n_estimators': 3000,
'max_depth': 13,
'min_child_weight': 39
}

In [ ]:
#referred to this note book (6)

from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
preds = []

kf = StratifiedKFold(n_splits=20,random_state=48,shuffle=True)
acc =[]
n=0 

for trn_idx, test_idx in kf.split(features, targets):
    X_tr, X_val = features.iloc[trn_idx], features.iloc[test_idx]
    y_tr,y_val= targets.iloc[trn_idx] , targets.iloc[test_idx]
    
    model = XGBClassifier(**params,objective= 'multi:softmax', tree_method='gpu_hist')
    model.fit(X_tr,y_tr,eval_set = [(X_val,y_val)],early_stopping_rounds =100,verbose =False)
    
    preds.append(model.predict(df_test))
    acc.append(accuracy_score(y_val,model.predict(X_val)))
    
    
    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)}")
    n+=1
    
    del X_tr,X_val,y_tr,y_val
    gc.collect()


In [ ]:
print(f"the mean Accuracy is : {round(np.mean(acc)*100,3)} ")

In [ ]:
prediction = stats.mode(preds)[0][0]

In [ ]:
predictions = encoder.inverse_transform(prediction)

In [ ]:
index = pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv")
index['Cover_Type'] = predictions
index.to_csv('submission.csv',index=False)